Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

=====> /Users/rob/.julia/dev/StatisticalRethinking/scripts/04



Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)


make: `/Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights' is up to date.
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE        MCSE      ESS
alpha 154.59475550 0.26815184 0.0042398528 0.00458902365 1000
 beta   0.90487783 0.04113430 0.0006503904 0.00056854831 1000
sigma   5.10869432 0.19588449 0.0030972057 0.00304761162 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%       97.5%   
alpha 154.07297500 154.4090000 154.5970000 154.776000 155.10605000
 beta   0.82378065   0.8767632   0.9053145   0.933414   0.98640965
sigma   4.73514125   4.9741500   5.1040100   5.239465   5.50568525



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.921 0.905809 5.3611; 154.122 0.896603 4.63607; … ; 155.353 0.937025 5.52065; 154.279 0.852106 4.89312]

Union{Missing, Float64}[154.496 0.982203 4.69179; 154.351 1.00646 4.6713; … ; 154.21 0.828923 5.13276; 154.989 0.936839 5.14297]

Union{Missing, Float64}[154.468 0.903715 4.82224; 154.803 0.911488 5.24619; … ; 154.506 0.889274 5.09102; 154.928 0.889314 5.42599]

Union{Missing, Float64}[154.745 0.961595 5.18245; 154.241 0.84892 5.1558; … ; 154.451 0.935599 4.91785; 154.721 0.881993 5.27434]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE        MCSE      ESS
alpha 154.59475550 0.26815184 0.0042398528 0.00458902365 1000
 beta   0.90487783 0.04113430 0.0006503904 0.00056854831 1000
sigma   5.10869432 0.19588449 0.0030972057 0.00304761162 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%       97.5%   
alpha 154.07297500 154.4090000 154.5970000 154.776000 155.10605000
 beta   0.82378065   0.8767632   0.9053145   0.933414   0.98640965
sigma   4.73514125   4.9741500   5.1040100   5.239465   5.50568525



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*